## Idea

$\textbf{Generating graph for arbitrary data }$

Given a set of data points $x_1,\dots, x_n$ and some notion of similarity $s_{ij} \geq 0$ (example: $s(x_i, x_j ) = \exp(\frac{−\|x_i − x_j \|^2}{2\sigma^2})$) between all pairs of data points $x_i$ and $x_j$ , the intuitive goal of clustering is to divide the data points into several groups such that points in the same group are similar and points in different groups are dissimilar to each other. In the corresponding graph there is no edge between pair of vertices is $s_{ij} = 0$ and, otherwise, there is an edge between $i,j$ with weight $s_{ij}$

Since we are not constructing a graph from the data, but just simply use given ones, we think of two vertices to be similar ($s_{ij}=1$) if there is an edge between them and zero otherwise.

$\textbf{Normalized and non-normalized Laplacians}$

Unnormalized Graph Laplacian (strong connection between connected components in a graph and the spectrum of Laplacian):

$$
L=D-A (\text{or} \ D-W  \ \text{in case of weighted graph})
$$

Normalized Laplacians:


$$
L_{sym} := D^{−1/2}LD^{−1/2} = I − D^{−1/2}W D^{−1/2}\\
L_{rw} :=D^{−1}L=I−D^{−1}W.
$$

$\textbf{Measuring sizes of subsets}$

Let $A_{1},\dots,A_{k}$ be some subsets of vertices. Two ways of measuring sizes of subsets are considered:

$$
|A_{i}| = \{\text{number of vertices in $A_{i}$}\}
$$

$$
vol(A_{i}) = \sum \limits_{i \in A_{i}}d_{i}
$$

where $d_{i}$ is the degree of vertex $i$.

$\textbf{Intuition behind spectral clustering algorithms}$

For given graph, the problem can be restated the following way: one wants to find a partition of the graph such that the edges between different groups have a very low weight (or there are not that many egdes between different groups) and the edges within a group have high weight (points within the same cluster are similar to each other and, hence, there are a lot of edges within the group). Spectral clustering can be derived as an approximation to such graph partitioning problems. Define for arbitrary sets $A,B$:

$$
W(A,B):= \sum \limits_{i \in A,j \in B}  w_{ij}.
$$

Hence, the simpliest way to get partition - solve mincut problem ($\overline{A}$ denotes complement of $A$):

$$
cut(A_1,\dots,A_k) = \frac{1}{2}\sum\limits_{i=1}^kW(A_i,\overline{A_i}) \rightarrow \min
$$

But in many cases the solution of mincut simply separates one individual vertex from the rest of the graph. To overcome such problems (to balance subsets) the following values are considered:

$$
RatioCut(A_1,\dots,A_k) = \sum \limits_{i=1}^k \frac{cut(A_i,\overline{A_i})}{|A_{i}|}
$$

$$
Ncut(A_1,\dots,A_k)= \sum \limits_{i=1}^k \frac{cut(A_i,\overline{A_i})}{vol(A_{i})}
$$

Unfortunately, introducing balancing conditions makes the previously simple to solve mincut problem become NP hard. Spectral clustering is a way to solve relaxed versions of those problems (relaxing RatioCut provides unnormalized spectral clustering, relaxing Ncut provides normalized spectral clustering with using $L_{rw}$).

Also spectral clustering can be interpreted as trying to find a partition of the graph such that the random walk stays long within the same cluster and seldom jumps between clusters (there is a connection with Ncut problem).

In [1]:
import numpy as np
from Lib.Load import download_graph, download_labels, save_graph, save_labels
from Lib.Metrics import compute_nmi, compute_recall, compute_precision, compute_avg_f1, compute_modularity, compute_normalized_cut
from Lib.Transformations import compute_clusters_from_labels, compute_adj_mat
from Lib.Algorithms import get_degree, get_nonnorm_lapl, get_lapl_sym,get_lapl_rw,unnorm_predict,lrw_predict,lsym_predict

## Unnormalized spectral clustering

Algorithm is the following (considering case when there are $k$ clusters and also when talking about eigenvalues consider the smallest ones):

$\cdot$ Given adjacency matrix $A$, compute its Laplacian $L$

$\cdot$ Compute the first $k$ eigenvectors $u_1, \dots , u_k$ of $L$. Let $U \in \mathbb{R}^{n\times k}$ be the matrix containing the vectors $u_1 , \dots, u_k$ as columns.

$\cdot$ For $i = 1,\dots,n$, let $y_i \in \mathbb{R}^k$ be the vector or the $i$-th row of $U$

$\cdot$ Cluster the points $(y_i),i=1,\dots,n$ with the $k$-means algorithm into clusters $C_1,...,C_k$

Now we simply check on our graphs

$\textbf{SBM}$

In [2]:
n_nodes, edge_list = download_graph('sbm')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('sbm')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = unnorm_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  1.0
Recall =  1.0
Precision =  1.0
F1 =  1.0
Modularity =  0.249741735537
Norm. cut =  0.500258397933


$\textbf{Karate}$

In [3]:
n_nodes, edge_list = download_graph('karate')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('karate')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = unnorm_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.455001600015
Recall =  0.8125
Precision =  0.875
F1 =  0.813186813187
Modularity =  0.233974358974
Norm. cut =  0.376068376068


$\textbf{Football}$

In [4]:
n_nodes, edge_list = download_graph('football')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('football')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = unnorm_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.92419578687
Recall =  0.889285714286
Precision =  0.873677248677
F1 =  0.886670290398
Modularity =  0.600516540747
Norm. cut =  4.04619502004


$\textbf{Polbooks}$

In [5]:
n_nodes, edge_list = download_graph('polbooks')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('polbooks')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = unnorm_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.651179091301
Recall =  0.722232363428
Precision =  0.839196310935
F1 =  0.733535851679
Modularity =  0.450527300867
Norm. cut =  0.616538296962


## Normalized spectral clustering with $L_{rw}$

Algorithm is the following (considering case when there are $k$ clusters and also when talking about eigenvalues consider the smallest ones):

$\cdot$ Given adjacency matrix $A$, compute its normalized Laplacian $L_{rw}$

$\cdot$ Compute the first $k$ eigenvectors $u_1, \dots , u_k$ of $L_{rw}$. Let $U \in \mathbb{R}^{n\times k}$ be the matrix containing the vectors $u_1 , \dots, u_k$ as columns.

$\cdot$ For $i = 1,\dots,n$, let $y_i \in \mathbb{R}^k$ be the vector or the $i$-th row of $U$

$\cdot$ Cluster the points $(y_i),i=1,\dots,n$ with the $k$-means algorithm into clusters $C_1,...,C_k$

Now we simply check on our graphs

$\textbf{SBM}$

In [6]:
n_nodes, edge_list = download_graph('sbm')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('sbm')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lrw_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  1.0
Recall =  1.0
Precision =  1.0
F1 =  1.0
Modularity =  0.249741735537
Norm. cut =  0.500258397933


$\textbf{Karate}$

In [7]:
n_nodes, edge_list = download_graph('karate')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('karate')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lrw_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.313617263711
Recall =  0.71875
Precision =  0.833333333333
F1 =  0.704347826087
Modularity =  0.192554240631
Norm. cut =  0.422764227642


$\textbf{Football}$

In [8]:
n_nodes, edge_list = download_graph('football')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('football')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lrw_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.92419578687
Recall =  0.889285714286
Precision =  0.873677248677
F1 =  0.886670290398
Modularity =  0.600516540747
Norm. cut =  4.04619502004


$\textbf{Polbooks}$

In [9]:
n_nodes, edge_list = download_graph('polbooks')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('polbooks')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lrw_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.623260331179
Recall =  0.724617575116
Precision =  0.750069803728
F1 =  0.732017982018
Modularity =  0.48448691646
Norm. cut =  0.414272054397


## Normalized spectral clustering with $L_{sym}$

Algorithm is the following (considering case when there are $k$ clusters and also when talking about eigenvalues consider the smallest ones):

$\cdot$ Given adjacency matrix $A$, compute its normalized Laplacian $L_{sym}$

$\cdot$ Compute the first $k$ eigenvectors $u_1, \dots , u_k$ of $L_{sym}$. Let $U \in \mathbb{R}^{n\times k}$ be the matrix containing the vectors $u_1 , \dots, u_k$ as columns.

$\cdot$ Form the matrix $T \in \mathbb{R}^{n\times k}$ from $U$ by normalizing the rows to norm 1

$\cdot$ For $i = 1,\dots,n$, let $t_i \in \mathbb{R}^k$ be the vector or the $i$-th row of $T$

$\cdot$ Cluster the points $(t_i),i=1,\dots,n$ with the $k$-means algorithm into clusters $C_1,...,C_k$

Now we simply check on our graphs

$\textbf{SBM}$

In [10]:
n_nodes, edge_list = download_graph('sbm')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('sbm')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lsym_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  1.0
Recall =  1.0
Precision =  1.0
F1 =  1.0
Modularity =  0.249741735537
Norm. cut =  0.500258397933


$\textbf{Karate}$

In [11]:
n_nodes, edge_list = download_graph('karate')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('karate')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lsym_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.836504088906
Recall =  0.96875
Precision =  0.973684210526
F1 =  0.970357454228
Modularity =  0.359960552268
Norm. cut =  0.262626262626


$\textbf{Football}$

In [12]:
n_nodes, edge_list = download_graph('football')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('football')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lsym_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.92419578687
Recall =  0.889285714286
Precision =  0.873677248677
F1 =  0.886670290398
Modularity =  0.600516540747
Norm. cut =  4.04619502004


$\textbf{Polbooks}$

In [13]:
n_nodes, edge_list = download_graph('polbooks')
A = compute_adj_mat(n_nodes, edge_list)

labels_true = download_labels('polbooks')
clusters_true = compute_clusters_from_labels(labels_true)

num_of_clusters = np.max(labels_true) + 1

labels_pred = lsym_predict(A, num_of_clusters)
clusters_pred = compute_clusters_from_labels(labels_pred)

print "NMI = ", compute_nmi(labels_true, labels_pred)
print "Recall = ", compute_recall(clusters_true, clusters_pred)
print "Precision = ", compute_precision(clusters_true, clusters_pred)
print "F1 = ", compute_avg_f1(clusters_true, clusters_pred)
print "Modularity = ", compute_modularity(labels_pred, edge_list)
print "Norm. cut = ", compute_normalized_cut(labels_pred, clusters_pred, edge_list)

NMI =  0.542276479422
Recall =  0.74093680406
Precision =  0.732554200542
F1 =  0.735050982118
Modularity =  0.501504002962
Norm. cut =  0.481903537479
